In this notebook, we have extracted a new dataset from the original one we got from RoboFlow, and the reason we did this step is that the original data classifies the objects in each image to be either a line or a car , which is not the case in our problem that we are trying to solve. To overcome this we have created the two classes that is tailored to our use case today which is classifing the car if it correctly parked or incorrectly parked , this way will help us getting better results and making the training process easier 

In [6]:
import tensorflow as tf
import json, os, shutil
from shapely.geometry import box 


In [ ]:
base_dir = "dataset" 
output_dir = "classification_dataset"
splits = ["train", "valid", "test"]


In [ ]:
def is_car_correct(car, lines):
    """
    Determine if a car is correctly parked given divider line boxes.
    car: [x1, y1, x2, y2]
    lines: list of [x1, y1, x2, y2]
    """

    # Assume lines are vertical dividers (x boundaries).
    # If they are horizontal, swap x/y in sorting below.
    line_x = sorted([ (l[0], l[2]) for l in lines ])  # collect x positions

    # Get car edges
    car_left, car_right = car[0], car[2]

    if len(line_x) == 0:
        # No lines = undefined → mark incorrect
        return False

    elif len(line_x) == 1:
        # One divider line → car must be completely left or right
        lx1, lx2 = line_x[0]
        divider_x = (lx1 + lx2) / 2
        if car_right <= divider_x or car_left >= divider_x:
            return True
        else:
            return False

    else:
        # Multiple lines → create slots between consecutive lines
        dividers = sorted([ (lx1+lx2)/2 for lx1, lx2 in line_x ])
        # Slots are between each pair of consecutive dividers
        slots = [(dividers[i], dividers[i+1]) for i in range(len(dividers)-1)]

        # Count how many slots the car fits in
        fits_in = 0
        for (s_left, s_right) in slots:
            if car_left >= s_left and car_right <= s_right:
                fits_in += 1

        return fits_in == 1  # must fit exactly one slot


In [1]:

# -------------------- MAIN LOOP --------------------
for split in splits:
    ann_file = os.path.join(base_dir, split, "_annotations.coco.json")
    img_dir = os.path.join(base_dir, split)

    out_correct = os.path.join(output_dir, split, "correct")
    out_incorrect = os.path.join(output_dir, split, "incorrect")
    os.makedirs(out_correct, exist_ok=True)
    os.makedirs(out_incorrect, exist_ok=True)

    with open(ann_file) as f:
        data = json.load(f)

    # Hardcoded IDs
    car_id = 1
    line_id = 2

    anns_by_image = {}
    for ann in data["annotations"]:
        anns_by_image.setdefault(ann["image_id"], []).append(ann)

    for img in data["images"]:
        img_id = img["id"]
        img_path = os.path.join(img_dir, img["file_name"])

        anns = anns_by_image.get(img_id, [])
        cars, lines = [], []

        for ann in anns:
            x, y, w, h = ann["bbox"]
            bbox = [x, y, x+w, y+h]
            if ann["category_id"] == car_id:
                cars.append(bbox)
            elif ann["category_id"] == line_id:
                lines.append(bbox)

        incorrect_flag = False
        for car in cars:
            if not is_car_correct(car, lines):
                incorrect_flag = True
                break

        if incorrect_flag:
            shutil.copy(img_path, out_incorrect)
        else:
            shutil.copy(img_path, out_correct)

NameError: name 'splits' is not defined

In [16]:
import shutil


In [17]:
shutil.make_archive("classification_dataset", 'zip', "classification_dataset")


'/Users/farahalhanaya/computer-vision-project-mawqif/classification_dataset.zip'